Name: Nguyễn Lâm Vĩnh Gia - ID: 2052074

# NAIVE BAYES CLASSIFIER

* Seperate data by the type of flower 

In [36]:
import pandas as pd
import numpy as np
from math import sqrt, pi, exp

# Function to load the CSV file and convert it to numpy array for further execution
def loadCSV(file_path):
    df = pd.read_csv(file_path, encoding = 'utf-8')
    df['variety'] = pd.Categorical(df['variety']).codes
    data_frame = df.to_numpy()
    return data_frame

# Function to seperate data by class
def seperateClass(data_frame):
    seperate_class = dict()
    for i in range(len(data_frame)):
        data_row = data_frame[i]
        class_name = data_row[-1]
        if class_name not in seperate_class:
            seperate_class[class_name] = list()
        seperate_class[class_name].append(data_row[:(len(data_row) - 1)])
    return seperate_class

file_path = r"C:\Studying Documents\Artificial intelligient\Exercise\Dataset\Iris.csv"
data_frame = loadCSV(file_path)
sep_class = seperateClass(data_frame)
print(sep_class)

{0.0: [array([5.1, 3.5, 1.4, 0.2]), array([4.9, 3. , 1.4, 0.2]), array([4.7, 3.2, 1.3, 0.2]), array([4.6, 3.1, 1.5, 0.2]), array([5. , 3.6, 1.4, 0.2]), array([5.4, 3.9, 1.7, 0.4]), array([4.6, 3.4, 1.4, 0.3]), array([5. , 3.4, 1.5, 0.2]), array([4.4, 2.9, 1.4, 0.2]), array([4.9, 3.1, 1.5, 0.1]), array([5.4, 3.7, 1.5, 0.2]), array([4.8, 3.4, 1.6, 0.2]), array([4.8, 3. , 1.4, 0.1]), array([4.3, 3. , 1.1, 0.1]), array([5.8, 4. , 1.2, 0.2]), array([5.7, 4.4, 1.5, 0.4]), array([5.4, 3.9, 1.3, 0.4]), array([5.1, 3.5, 1.4, 0.3]), array([5.7, 3.8, 1.7, 0.3]), array([5.1, 3.8, 1.5, 0.3]), array([5.4, 3.4, 1.7, 0.2]), array([5.1, 3.7, 1.5, 0.4]), array([4.6, 3.6, 1. , 0.2]), array([5.1, 3.3, 1.7, 0.5]), array([4.8, 3.4, 1.9, 0.2]), array([5. , 3. , 1.6, 0.2]), array([5. , 3.4, 1.6, 0.4]), array([5.2, 3.5, 1.5, 0.2]), array([5.2, 3.4, 1.4, 0.2]), array([4.7, 3.2, 1.6, 0.2]), array([4.8, 3.1, 1.6, 0.2]), array([5.4, 3.4, 1.5, 0.4]), array([5.2, 4.1, 1.5, 0.1]), array([5.5, 4.2, 1.4, 0.2]), array([

* Summarize the dataset to get the mean, standard deviation and length of each data column

In [32]:
# Function to calculate the mean of the data list
def mean(data_list):
    return sum(data_list) / float(len(data_list))

# Function to calculate the standard deviation of the data list
def stdev(data_list):
    data_mean = mean(data_list)
    variance = sum((x - data_mean) ** 2 for x in data_list) / float(len(data_list) - 1)
    return sqrt(variance)

# Function to summarize the dataset
def dataSummarize(data_frame):
    summary = [(mean(column), stdev(column), len(column)) for column in zip(*data_frame)]
    del(summary[-1])
    return summary

data_summary = dataSummarize(data_frame)
print(data_summary)

[(5.843333333333335, 0.8280661279778629, 150), (3.057333333333334, 0.435866284936698, 150), (3.7580000000000027, 1.7652982332594667, 150), (1.199333333333334, 0.7622376689603465, 150)]


* Summarize the data of each type of flower

In [33]:
def summarize_byClass(data_frame):
    sep_class = seperateClass(data_frame)
    summary_by_class = dict()
    for class_name, data_rows in sep_class.items():
        summary_by_class[class_name] = dataSummarize(data_rows)
    return summary_by_class

lookup_table = summarize_byClass(data_frame)
print(lookup_table)

{0.0: [(5.005999999999999, 0.3524896872134512, 50), (3.428000000000001, 0.3790643690962886, 50), (1.4620000000000002, 0.1736639964801841, 50)], 1.0: [(5.936, 0.5161711470638635, 50), (2.7700000000000005, 0.3137983233784114, 50), (4.26, 0.46991097723995806, 50)], 2.0: [(6.587999999999998, 0.635879593274432, 50), (2.9739999999999998, 0.3224966381726376, 50), (5.552, 0.5518946956639835, 50)]}


* Calculate the normal distribution of each data point 

In [35]:
# Function to calcultae the normal (Gaussian) probability of a data point
def gaussianProbability(x, mu, sigma):
    return (1 / (sigma * sqrt(2 * pi))) * exp(- (0.5 * ((x - mu) / sigma) ** 2))

print(gaussianProbability(3.4, 3.0, 2.0))

0.19552134698772797


* Calculate the probaility of belonging in a specific class of a data point

    We know that the classical naive Bayesian formula is

<h1><center>$ p(B|A) = \frac{p(A|B)p(B)}{p(A)} $</center></h1>

which those probabilities share the same denominator. Besides, as we are only interested in the problem of classification, we will simplify the calculation of the probability by eliminate the calculation of the denominator in the Bayes's formula

In [41]:
# Function to calculate the probability of belonging to each class of data of a data point
def classProbability(summary, data_point):
    data_length = sum([summary[class_][0][2] for class_ in summary])
    class_prob = dict()
    for class_name, class_summaries in summary.items():
        # Get the probability of each class with respect to the whole data frame
        class_prob[class_name] = summary[class_name][0][2] / float(data_length)
        # Calculate the probability of belonging in each data class of the new data point
        for i in range(len(class_summaries)):
            mean, stdev, num = class_summaries[i]
            class_prob[class_name] *= gaussianProbability(data_point[i], mean, stdev)
    return class_prob

# Check the result of a random data point
print(classProbability(lookup_table, data_frame[10]))         

{0.0: 0.36858194403030686, 1.0: 6.481447668566026e-11, 2.0: 5.107275067804685e-15}


From the above result, we can easily see that the probability of the data point belonging to the first class is highest, which mean this data belongs to the Setosa flower.

## The OOP version of the Naive Bayes algorithm

In [50]:
import pandas as pd
import numpy as np
from math import sqrt, pi, exp

# Class of Naive Bayes for Iris-alike dataset
class NaiveBayes():
    # Constructor of the class
    def __init__(self, file_path):
        self.__file_path = file_path
        df = pd.read_csv(self.__file_path, encoding = 'utf-8')
        # Convert the class type into integer for easier manipulation
        df['variety'] = pd.Categorical(df['variety']).codes
        self.data_frame = df.to_numpy()
        
    # Function to seperate data by class
    def seperateClass(self):
        seperate_class = dict()
        for i in range(len(self.data_frame)):
            data_row = self.data_frame[i]
            class_name = data_row[-1]
            if class_name not in seperate_class:
                seperate_class[class_name] = list()
            seperate_class[class_name].append(data_row[:(len(data_row) - 1)])
        return seperate_class
    
    # Function to calculate the mean of the data list
    def mean(self, data_list):
        return sum(data_list) / float(len(data_list))

    # Function to calculate the standard deviation of the data list
    def stdev(self, data_list):
        data_mean = self.mean(data_list)
        variance = sum((x - data_mean) ** 2 for x in data_list) / float(len(data_list) - 1)
        return sqrt(variance)

    # Function to summarize the dataset
    def dataSummarize(self, data_frame):
        summary = [(self.mean(column), self.stdev(column), len(column)) for column in zip(*data_frame)]
        del(summary[-1])
        return summary
    
    # Function to sumarize the data set by the class 
    def summarize_byClass(self):
        sep_class = self.seperateClass()
        summary_by_class = dict()
        for class_name, data_rows in sep_class.items():
            summary_by_class[class_name] = self.dataSummarize(data_rows)
        return summary_by_class

    # Function to calcultae the normal (Gaussian) probability of a data point
    def gaussianProbability(self, x, mu, sigma):
        return (1 / (sigma * sqrt(2 * pi))) * exp(- (0.5 * ((x - mu) / sigma) ** 2))
    
    # Function to calculate the probability of belonging to each class of data of a data point
    def classProbability(self, data_point):
        summary = self.summarize_byClass()
        data_length = sum([summary[class_][0][2] for class_ in summary])
        class_prob = dict()
        for class_name, class_summaries in summary.items():
            # Get the probability of each class with respect to the whole data frame
            class_prob[class_name] = summary[class_name][0][2] / float(data_length)
            # Calculate the probability of belonging in each data class of the new data point
            for i in range(len(class_summaries)):
                mean, stdev, num = class_summaries[i]
                class_prob[class_name] *= self.gaussianProbability(data_point[i], mean, stdev)
        return class_prob
    
    # Function to classify the class of the data point
    def bayesClassify(self, data_point):
        max_prob = 0.0
        max_class = 0.0
        class_prob = self.classProbability(data_point)
        for classes in class_prob:
            if class_prob[classes] > max_prob:
                max_prob = class_prob[classes]
                max_class = classes
        return max_class

if __name__ == '__main__':
    file_path = r"C:\Studying Documents\Artificial intelligient\Exercise\Dataset\Iris.csv"
    bayes_classifier = NaiveBayes(file_path)
    flower_1 = bayes_classifier.bayesClassify([4.7, 3, 2, 2.5])
    flower_2 = bayes_classifier.bayesClassify([5.9, 3, 5.1, 1.8])
    print("The class of the 1st data point is", str(flower_1))
    print("The class of the 2nd data point is", str(flower_2))

The class of the 1st data point is 0.0
The class of the 2nd data point is 2.0
